In [103]:
pip install numpy pandas yfinance

Note: you may need to restart the kernel to use updated packages.


In [104]:
import numpy as np
import pandas as pd
import yfinance as yf

In [105]:
goog = yf.Ticker("GOOG")
google_five_year_history = goog.history(period='5y')

Why this is a "Full Stack" Quant Project
np.random.normal & np.cumsum: You demonstrated you understand stochastic processes (random walks), which is the foundation of option pricing.

pd.date_range: You handled time-indexing, essential for time-series alignment.

rolling() & shift(): These are the two most frequently used Pandas functions in finance. You used shift() correctly to avoid Look-Ahead Bias, a critical error junior quants make.

np.where: You avoided slow Python loops for trade logic.

cummax(): You calculated Drawdown, which is the #1 metric risk managers look at (even more than returns).

In [106]:
google_five_year_history

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-02-16 00:00:00-05:00,104.434728,106.832743,104.434728,105.305199,22676000,0.0,0.0
2021-02-17 00:00:00-05:00,104.218347,105.888817,104.164748,105.623306,21418000,0.0,0.0
2021-02-18 00:00:00-05:00,104.733988,105.842919,104.402474,105.071953,22432000,0.0,0.0
2021-02-19 00:00:00-05:00,105.174679,105.733484,104.089813,104.274925,29148000,0.0,0.0
2021-02-22 00:00:00-05:00,102.580624,103.792535,102.339437,102.475418,27350000,0.0,0.0
...,...,...,...,...,...,...,...
2026-02-09 00:00:00-05:00,321.239990,327.739990,317.700012,324.399994,26103300,0.0,0.0
2026-02-10 00:00:00-05:00,320.910004,321.630005,314.660004,318.630005,25281900,0.0,0.0
2026-02-11 00:00:00-05:00,318.959991,321.045013,310.130005,311.329987,24008100,0.0,0.0


In [107]:
# Rolling Window Calculations (The bread and butter of Pandas)
window = 20
google_five_year_history['SMA'] = google_five_year_history['Close'].rolling(window=window).mean()       # Simple Moving Average
google_five_year_history['Vol'] = google_five_year_history['Close'].std()        # Rolling Volatility

In [108]:
# Calculate Log Returns (Preferred in quant finance over simple returns)
# np.log -> Vectorized math function
# shift(1) -> Aligns today's price with yesterday's, as such you are able to calculate the log returns by alligning todays price with yesterdays price
google_five_year_history['Log_Ret'] = np.log(google_five_year_history['Close']/ google_five_year_history['Close'].shift(1))

In [109]:
# Vectorized Z-Score Calculation (NumPy Broadcasting)
# Logic: How many standard deviations is price away from the mean?
google_five_year_history['Z Score'] = (google_five_year_history['Close'] - google_five_year_history['SMA']) / google_five_year_history['Vol']

In [110]:
# np.where is faster than applying a function with if/else
# Strategy: Long (+1) if Z < -2 (Oversold), Short (-1) if Z > 2 (Overbought), else Flat (0)
google_five_year_history['Position'] = np.where(google_five_year_history['Z Score'] < 0, 1, 
                 np.where(google_five_year_history['Z Score'] > 0, -1, 0))

In [111]:
# --- 5. BACKTESTING: Alignment & Performance ---

# Look-ahead Bias Prevention: We must shift the position down by 1 day.
# We make the decision Today based on Today's close, so we earn returns Tomorrow.
google_five_year_history['Strategy_Ret'] = google_five_year_history['Position'].shift(1) * google_five_year_history['Log_Ret']

# Cumulative Returns (Equity Curve)
google_five_year_history['Cum_Ret'] = google_five_year_history['Strategy_Ret'].cumsum().apply(np.exp)

# Drawdown Calculation (Crucial Risk Metric)
# cummax() tracks the highest peak reached so far
running_max = google_five_year_history['Cum_Ret'].cummax()
google_five_year_history['Drawdown'] = (google_five_year_history['Cum_Ret'] - running_max) / running_max

In [112]:
display(google_five_year_history)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,SMA,Vol,Log_Ret,Z Score,Position,Strategy_Ret,Cum_Ret,Drawdown
Date,,,,,,,,,,,,,,,
2021-02-16 00:00:00-05:00,104.434728,106.832743,104.434728,105.305199,22676000,0.0,0.0,NaN,52.823323,NaN,NaN,0,NaN,NaN,NaN
2021-02-17 00:00:00-05:00,104.218347,105.888817,104.164748,105.623306,21418000,0.0,0.0,NaN,52.823323,0.003016,NaN,0,0.000000,1.000000,0.000000
2021-02-18 00:00:00-05:00,104.733988,105.842919,104.402474,105.071953,22432000,0.0,0.0,NaN,52.823323,-0.005234,NaN,0,-0.000000,1.000000,0.000000
2021-02-19 00:00:00-05:00,105.174679,105.733484,104.089813,104.274925,29148000,0.0,0.0,NaN,52.823323,-0.007614,NaN,0,-0.000000,1.000000,0.000000
2021-02-22 00:00:00-05:00,102.580624,103.792535,102.339437,102.475418,27350000,0.0,0.0,NaN,52.823323,-0.017408,NaN,0,-0.000000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-02-09 00:00:00-05:00,321.239990,327.739990,317.700012,324.399994,26103300,0.0,0.0,332.930499,52.823323,0.004015,-0.161491,1,0.004015,0.581807,-0.500356
2026-02-10 00:00:00-05:00,320.910004,321.630005,314.660004,318.630005,25281900,0.0,0.0,332.225499,52.823323,-0.017947,-0.257377,1,-0.017947,0.571459,-0.509243
2026-02-11 00:00:00-05:00,318.959991,321.045013,310.130005,311.329987,24008100,0.0,0.0,330.970499,52.823323,-0.023177,-0.371815,1,-0.023177,0.558367,-0.520486


In [115]:
clean_df = google_five_year_history.dropna()

print(f"Final Portfolio Value: {clean_df['Cum_Ret'].iloc[-1]:.4f}")
print(f"Max Drawdown: {clean_df['Drawdown'].min():.2%}")
print(f"Sharpe Ratio: {(clean_df['Strategy_Ret'].mean() / clean_df['Strategy_Ret'].std()) * np.sqrt(252):.2f}")

# Check the first few rows to see the structure
print("\nData Snapshot:")
print(clean_df[['Close', 'SMA', 'Z Score', 'Position']].tail())

Final Portfolio Value: 0.5467
Max Drawdown: -53.05%
Sharpe Ratio: -0.40

Data Snapshot:
                                Close         SMA   Z Score  Position
Date                                                                 
2026-02-09 00:00:00-05:00  324.399994  332.930499 -0.161491         1
2026-02-10 00:00:00-05:00  318.630005  332.225499 -0.257377         1
2026-02-11 00:00:00-05:00  311.329987  330.970499 -0.371815         1
2026-02-12 00:00:00-05:00  309.369995  329.623499 -0.383420         1
2026-02-13 00:00:00-05:00  304.829987  328.206998 -0.442551         1
